In [1]:
import gym
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

#for text processing
import spacy
import re
import pandas as pd


In [3]:
env = gym.make("Taxi-v3").env #Taxi-v2 is not available now.


In [4]:

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



#### There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions."

### Fetching Origing, Destination, and Time of Pickup from the sms data 

In [5]:
import re
import pandas as pd
import numpy as np


def fetch_pickup_drop(line):
    city = pd.read_csv("./city.csv", header = None)
    city_list = list(city[0].values)
    r1 = re.findall(r"[\d+]+ AM", line)
    if len(r1):
        time = r1[0].strip()
    else:
        r1 = re.findall(r"[\d+]+ PM", line)
        time = r1[0].strip()
        
    time_of_pickup = time        
    ##Fetching Origin and Destination
    candidate_list = []
    for city in city_list:
        if city in line:
            candidate_list.append(city)

    if len(candidate_list) == 2:
        flag = 0
        for city in candidate_list:
            pattern1 = 'to '+city
            pattern2 = 'for '+city
            r1 = re.findall(pattern1, line)
            r2 = re.findall(pattern2, line)
            if len(r1) or len(r2):
                destination = city
                candidate_list.remove(city)
                origin = candidate_list[0]
                flag =1
                break
        return[origin, destination, time_of_pickup]

        if flag == 0:
            print("Destination not found for patter1", pattern1)
            print("Destination not found for patter2", pattern1)
            print("line is", line)
            return [origin, destination, time_of_pickup]

    else:
        print("Does not include both origin and destination")
        origin = 'NA'
        destination = 'NA'
        return [origin, destination, time_of_pickup]


In [6]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


## Summing up the Q-Learning Process
Breaking it down into steps, we get

Initialize the Q-table by all zeros.

Start exploring actions: 

For each state, select any one among all possible actions for the current state (S).

Travel to the next state (S') as a result of that action (a).

For all possible actions from the state (S') select the one with the highest Q-value.

Update Q-table values using the equation.

Set the next state as the current state.

If goal state is reached, then end and repeat the process.


## Exploiting learned values
After enough random exploration of actions, the Q-values tend to converge serving our agent as an action-value function which it can exploit to pick the most optimal action from a given state.

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called ϵ "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [7]:
import numpy as np

#Initialize Q_table
q_table = np.zeros([env.observation_space.n, env.action_space.n])


In [8]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001): #if reduced the range, then agent will stuck in loop not sure what was happening though.
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # explore
        else:
            action = np.argmax(q_table[state]) # exploit

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

np.save("./q_table.npy", q_table)

Wall time: 2min 43s


In [9]:
#Load trained q_table for evaluation

q_table = np.load("./q_table.npy")

In [10]:
def create_loc_dict(city_df):
    loc_dict = {}
    ## Create dictionary example, loc_dict['dwarka sector 23] = 0
    for i in range(len(city_df)):
        loc_dict.update({city_df.iloc[i].location:city_df.iloc[i].mapping})
        
    return loc_dict

In [11]:
def check_pick_up_drop_correction(pick_up, drop, line_num):
    orig_df = pd.read_csv("./org_df.csv")
    original_origin = orig_df.iloc[line_num]['origin']
    original_destination = orig_df.iloc[line_num]['dest']
    if original_origin == pick_up and original_destination == drop:
        return True
    else:
        return False

In [12]:
"""Evaluate agent's performance after Q-learning"""

# 1) We need to take text drom "sms.txt" and fetch pickup and drop from it.
# 2) Generate the random state from an enviroment and change the pick up and drop as the fetched one
# 3) Evaluate you q_table performance on all the texts given in sms.txt.
# 4) Have a check if the fetched pickup, drop is not matching with original pickup, drop using orig.csv
# 5) If fetched pickup or/and drop does not match with the original, add penality and reward -10
# 6) Calculate the Total reward, penalities, Wrong pickup/drop predicted and Average time steps per episode.

total_epochs, total_penalties, total_reward, wrong_predictions = 0, 0, 0, 0


count = 0
time_list = []
f = open("./sms.txt", "r")
num_of_lines = 1000
city = pd.read_csv("./city.csv")

loc_dict = create_loc_dict(city)
line_num = 0
for line in f:
    pickup, drop, time = fetch_pickup_drop(line)
    pickUpIdx  = loc_dict[pickup]
    dropIdx = loc_dict[drop]
    env.reset()
    #print(line_num)
    decision = check_pick_up_drop_correction(pickup,drop,line_num)
    
    if not decision:
        total_penalities += 1
        reward = -10
        total_reward += reward
        wrong_predictions += 1
    
    episodes = 10

    for _ in range(episodes):
        state = env.reset()
        epochs, penalties, reward = 0, 0, 0
        
        done = False
        env.encode(np.random.randint(0,4),np.random.randint(0,4),pickUpIdx,dropIdx)
        
        while not done:
            action = np.argmax(q_table[state])
            state, reward, done, info = env.step(action)
            #print(state,reward,done," ",line_num,episodes)
            #if reward == 20:
             #   print("20$ reward")
            if reward == -10:
                penalties += 1
            epochs += 1
    
        total_penalties += penalties
        total_epochs += epochs
        total_reward += reward
    
    line_num = line_num + 1

print(f"Results after {num_of_lines} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_of_lines}")
print(f"Average penalties per episode: {total_penalties / num_of_lines}")
print(f"Total number of wrong predictions", wrong_predictions)
print("Total Reward is", total_reward)

Results after 1000 episodes:
Average timesteps per episode: 130.914
Average penalties per episode: 0.0
Total number of wrong predictions 0
Total Reward is 200000
